In [3]:
from __future__ import print_function

import pandas as pd
import os
from dotenv import load_dotenv
from datetime import date, datetime, timedelta
import sqlalchemy
import pymysql
import openpyxl

In [3]:
load_dotenv()

(host, user, password, port, database) = (os.getenv('host'), os.getenv('user'), os.getenv('passwd'), os.getenv('port'), os.getenv('database'))

In [102]:
def get_connection():
    return sqlalchemy.create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )

In [103]:
engine = get_connection()

table = "cpt_codes"

In [104]:
show_table_query = "DESCRIBE " + table
with engine.connect() as conn:
    result = conn.execute(show_table_query)
for row in result:
    print(row)

('key', 'int', 'NO', 'PRI', None, 'auto_increment')
('id', 'varchar(64)', 'NO', '', None, '')
('code', 'varchar(16)', 'NO', 'UNI', None, '')
('type', 'varchar(32)', 'NO', '', None, '')
('proprietary_name', 'varchar(256)', 'NO', '', None, '')
('proprietary_name_suffix', 'varchar(128)', 'YES', '', None, '')
('nonproprietary_name', 'varchar(512)', 'YES', '', None, '')
('dosage_form', 'varchar(64)', 'YES', '', None, '')
('route', 'varchar(128)', 'YES', '', None, '')
('start_marketing_date', 'date', 'YES', '', None, '')
('end_marketing_date', 'date', 'YES', '', None, '')
('marketing_category_name', 'varchar(45)', 'YES', '', None, '')
('application_number', 'varchar(20)', 'YES', '', None, '')
('labeler_name', 'varchar(128)', 'YES', '', None, '')
('substance_name', 'varchar(3450)', 'YES', '', None, '')
('active_numenator_strength', 'varchar(510)', 'YES', '', None, '')
('active_ingredient_unit', 'varchar(1500)', 'YES', '', None, '')
('pharma_class', 'varchar(4010)', 'YES', '', None, '')
('desc

In [8]:
cpt_path = 'ndcxls/PPRRVU23_JAN.xlsx'
start_row = 9
table_ndc = 'ndc_product'
columns = ['code', 'description']
sqltypes_ndc = {'code': sqlalchemy.types.VARCHAR(length=8), 'description': sqlalchemy.types.VARCHAR(length=30)}

df = pd.read_excel(cpt_path, names=columns,usecols=[0,2], skiprows= start_row)
df.columns = columns

In [9]:
print(df.code.str.len().max())
print(df.description.str.len().max())

7
28.0


In [10]:
df.dtypes

code           object
description    object
dtype: object

In [111]:
df.to_sql(name=table_ndc,con=engine, if_exists='append', index=False, chunksize=100, dtype=sqltypes_ndc)

96547